In [4]:
import pandas as pd
import os
import numpy as np
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.utils import shuffle

file_name = 'fxReynoldsV3.xlsx'
file_path = os.path.join(os.getcwd(), file_name)

ANNDF = pd.read_excel(file_path)

X = ANNDF.iloc[:, [1, 3]].values
Y = ANNDF.iloc[:, 2].values.reshape(-1, 1)

X, Y = shuffle(X, Y)

xtrain, xtest, ytrain, ytest = train_test_split(X, Y, train_size=0.75)  # 30% for testing and shuffle

scalerx = StandardScaler()
scalery = StandardScaler()

Xtrain = scalerx.fit_transform(xtrain)
Ytrain = scalery.fit_transform(ytrain)

mlp = MLPRegressor(max_iter=5000)

nneurons = [1, 2, 4, 8, 16, 32, 64, 128]
parameters = {'hidden_layer_sizes': [],
              'activation': ['relu', 'tanh', 'logistic'],
              'solver': ['adam']
}

for n in nneurons:
    parameters['hidden_layer_sizes'].append((n,))

for n1 in nneurons:
    for n2 in nneurons:
        parameters['hidden_layer_sizes'].append((n1, n2))

kf = KFold(n_splits=10, shuffle=True)

gsearchcv = GridSearchCV(estimator=mlp, param_grid=parameters, cv=kf, verbose=2)
gsearchcv.fit(Xtrain, Ytrain.ravel())

bestac = gsearchcv.best_score_ * 100  # Convert to percentage
bestparam = gsearchcv.best_params_

print('Best Accuracy: {:.2f} %'.format(bestac))
print('Best Parameters:', bestparam)

Xtest_scaled = scalerx.transform(xtest)
ypred = gsearchcv.predict(Xtest_scaled).reshape(-1, 1)
ypred_or = scalery.inverse_transform(ypred)

r2 = r2_score(ytest, ypred_or)
rmse = mean_squared_error(ytest, ypred_or, squared=False)

r2, rmse

Fitting 10 folds for each of 216 candidates, totalling 2160 fits
[CV] END activation=relu, hidden_layer_sizes=(1,), solver=adam; total time=   0.1s
[CV] END activation=relu, hidden_layer_sizes=(1,), solver=adam; total time=   0.1s
[CV] END activation=relu, hidden_layer_sizes=(1,), solver=adam; total time=   0.5s
[CV] END activation=relu, hidden_layer_sizes=(1,), solver=adam; total time=   0.3s
[CV] END activation=relu, hidden_layer_sizes=(1,), solver=adam; total time=   0.1s
[CV] END activation=relu, hidden_layer_sizes=(1,), solver=adam; total time=   0.3s
[CV] END activation=relu, hidden_layer_sizes=(1,), solver=adam; total time=   0.1s
[CV] END activation=relu, hidden_layer_sizes=(1,), solver=adam; total time=   0.5s
[CV] END activation=relu, hidden_layer_sizes=(1,), solver=adam; total time=   0.0s
[CV] END activation=relu, hidden_layer_sizes=(1,), solver=adam; total time=   0.0s
[CV] END activation=relu, hidden_layer_sizes=(2,), solver=adam; total time=   0.2s
[CV] END activation=re

(0.9600871097499866, 0.021110974097123238)

In [ ]:
print()

In [5]:
def cor1(Re):
    f = 2.344 * np.power(Re,-0.081) #correlação kaixin yan  300 < RE <3100
    return f

def cor2(Re):
    f = 2.765 * np.power(Re,-0.166) #correlação kaixin yan  300 < RE <3100
    return f

Recor1 = np.linspace(1000,3100,15)
stpkyan = [0.3745] * len(Recor1)

In [30]:
#EXPDATA
file_name = 'fxReynoldsV3.xlsx'
file_path = os.path.join(os.getcwd(), file_name)

dfexp = pd.read_excel(file_path, sheet_name='expdata')
reexp = dfexp.iloc[:,0].values
fexp = dfexp.iloc[:,1].values
stdpexp = dfexp.iloc[:,2].values


Xexp = dfexp.iloc[:,[0,2]].values
Yexp = dfexp.iloc[:,1].values

Xexp,Yexp = shuffle(Xexp,Yexp) 


xexptrain, xexptest, yexptrain, yexptest = train_test_split(Xexp, Yexp, train_size=0.75)



In [31]:
import numpy as np
import plotly.graph_objs as go
import plotly.offline as pyo





# Definir os intervalos para Re e Stdp
Re_vals = np.linspace(300, 10000, 100)  # Valores de Reynolds de 1000 a 8000
Stdp_vals = np.linspace(0.2992, 0.4331, 100)  # Valores de Stdp de 0.3326 a 0.4331

# Criar uma grade de Re e Stdp
Re_grid, Stdp_grid = np.meshgrid(Re_vals, Stdp_vals)
grid_points = np.c_[Re_grid.ravel(), Stdp_grid.ravel()]

# Prever os valores de 'f' usando a ANN nos pontos da grade
f_pred_scaled = gsearchcv.predict(scalerx.transform(grid_points))
f_pred = scalery.inverse_transform(f_pred_scaled.reshape(-1, 1))
f_pred_grid = f_pred.reshape(Re_grid.shape)

# Calcular a superfície usando a fórmula fornecida
#f_formula = 2.9728 + -6.0736e-05*Re_grid + -10.097*Stdp_grid + 5.0583e-09*Re_grid**2 + -4.1042e-05*Re_grid * Stdp_grid + 11.001*Stdp_grid**2

# Criar o gráfico 3D para os resultados da ANN com contornos
surface_ann = go.Surface(
    x=Re_grid, y=Stdp_grid, z=f_pred_grid, 
    colorscale='Viridis',
    name='Superfície ANN',
    colorbar=dict(title='ANN f', len=0.5, x=0.9),
    contours={
        "z": {
            "show": True,
            "start": np.min(f_pred_grid),
            "end": np.max(f_pred_grid),
            "size": (np.max(f_pred_grid) - np.min(f_pred_grid)) / 20,  # Ajustar o espaçamento dos contornos
            "color": "black",
        }
    }
)

# Criar o gráfico 3D para a fórmula fornecida com contornos
'''surface_formula = go.Surface(
    x=Re_grid, y=Stdp_grid, z=f_formula, 
    colorscale='Plasma',
    name='Superfície Fórmula',
    opacity=0.7,
    colorbar=dict(title='Fórmula f', len=0.5, x=1.05),
    contours={
        "z": {
            "show": True,
            "start": np.min(f_formula),
            "end": np.max(f_formula),
            "size": (np.max(f_formula) - np.min(f_formula)) / 20,
            "color": "black",
        }
    }
)'''

# Scatter plot dos valores originais
scatter_cor = go.Scatter3d(
    x=Recor1,  # Valores de Reynolds
    y=stpkyan,  # Valores de Stdp
    z=cor2(Recor1),  # Valores de 'f'
    mode='markers',
    marker=dict(size=5, color='red'),
    name='Valores Kaixin Yan'
)


# Scatter plot dos valores originais
scatter_data = go.Scatter3d(
    x=xexptest[:,0],  # Valores de Reynolds
    y=xexptest[:,1],  # Valores de Stdp
    z=yexptest,  # Valores de 'f'
    mode='markers',
    marker=dict(size=5, color='red'),
    name='Valores Originais'
)

# Definir o layout do gráfico
layout = go.Layout(
    title='Superfície de Resultados da ANN e da Fórmula com Valores Originais',
    scene=dict(
        xaxis=dict(title='Reynolds Number (Re)', backgroundcolor='white', gridcolor='lightgrey'),
        yaxis=dict(title='Stdp', backgroundcolor='white', gridcolor='lightgrey'),
        zaxis=dict(title='f', backgroundcolor='white', gridcolor='lightgrey'),
        bgcolor='white'
    ),
    paper_bgcolor='white',
    plot_bgcolor='white',
    showlegend=True
)

# Criar a figura com ambas as superfícies e o scatter plot
fig = go.Figure(data=[surface_ann ,scatter_cor, scatter_data], layout=layout)

# Mostrar o gráfico
pyo.plot(fig)


'temp-plot.html'